#TODO: Calculate the cos_distance of the train set and test set
#Save the data into files


In [12]:
import pandas as pd
import numpy as np
import math
import theano
from sklearn import datasets
import pylab
import theano.tensor as T

#load embedding
embedding = pd.read_csv('embedding.csv')
embedding.index = embedding['Unnamed: 0'].tolist()
del embedding['Unnamed: 0']


#load training context
train_context_f = open('coordinate_data_150/train/context.txt')
train_context_input = train_context_f.readlines()
n = 0
train_words = [(["unknown"]*6) for i in range(int(len(train_context_input)/3))]
train_types = [(["unknown"]*6) for i in range(int(len(train_context_input)/3))]
words_index = 0
types_index = 0
while n < len(train_context_input):
    t = train_context_input[n].split()
    if n%3==0:
        for i in range(0,6):
            train_words[words_index][i] = t[i]
        words_index = words_index + 1
    elif n%3==1:
        for i in range(0,6):                                
            train_types[types_index][i] = t[i]
        types_index = types_index + 1
    n = n + 1
    
#load training labels
train_label_f = open('coordinate_data_150/train/labels.txt')
train_label_input = train_label_f.readlines()
n = 0
train_labels = [([-1]*3) for i in range(len(train_label_input))]
while n < len(train_label_input):
    t = train_label_input[n].split()
    for i in range(0,3):
        train_labels[n][i] = int(t[i])
    n = n + 1
    
    
    
#计算余弦距离的函数
def cosine_dis(vector1,vector2):
    '''
    @vector1 and @vector2 are two input vectors
    @Return the cosine distance between two vectors
    '''
    mm = 0
    for i in range(0,len(vector1)):
        mm = mm + vector1[i]*vector2[i]
    dd1 = 0
    dd2 = 0
    for i in range(0,len(vector1)):
        dd1 = dd1 + vector1[i]*vector1[i]
        dd2 = dd2 + vector2[i]*vector2[i]
    dd1 = math.sqrt(dd1)
    dd2 = math.sqrt(dd2)
    result = mm/(dd1*dd2)
    return result

train_num = len(train_words); #训练量
nn_input_dim=15 #输入神经元个数  

#第一步：单纯以context中对应位置的词性作为训练标准
#计算context对应位置的余弦距离（2-4，3-5） TODO:改写这部分

train_type_dis = [([float]*nn_input_dim) for i in range(train_num)]
train_words_dis = [([float]*nn_input_dim) for i in range(train_num)]

for i in range(0,train_num):

    train_type_dis[i][0] = cosine_dis(embedding.ix[train_types[i][0]],embedding.ix[train_types[i][1]])
    train_type_dis[i][1] = cosine_dis(embedding.ix[train_types[i][0]],embedding.ix[train_types[i][2]])
    train_type_dis[i][2] = cosine_dis(embedding.ix[train_types[i][0]],embedding.ix[train_types[i][3]])
    train_type_dis[i][3] = cosine_dis(embedding.ix[train_types[i][0]],embedding.ix[train_types[i][4]])
    train_type_dis[i][4] = cosine_dis(embedding.ix[train_types[i][0]],embedding.ix[train_types[i][5]])
    train_type_dis[i][5] = cosine_dis(embedding.ix[train_types[i][1]],embedding.ix[train_types[i][2]])
    train_type_dis[i][6] = cosine_dis(embedding.ix[train_types[i][1]],embedding.ix[train_types[i][3]])
    train_type_dis[i][7] = cosine_dis(embedding.ix[train_types[i][1]],embedding.ix[train_types[i][4]])
    train_type_dis[i][8] = cosine_dis(embedding.ix[train_types[i][1]],embedding.ix[train_types[i][5]])
    train_type_dis[i][9] = cosine_dis(embedding.ix[train_types[i][2]],embedding.ix[train_types[i][3]])
    train_type_dis[i][10] = cosine_dis(embedding.ix[train_types[i][2]],embedding.ix[train_types[i][4]])
    train_type_dis[i][11] = cosine_dis(embedding.ix[train_types[i][2]],embedding.ix[train_types[i][5]])
    train_type_dis[i][12] = cosine_dis(embedding.ix[train_types[i][3]],embedding.ix[train_types[i][4]])
    train_type_dis[i][13] = cosine_dis(embedding.ix[train_types[i][3]],embedding.ix[train_types[i][5]])
    train_type_dis[i][14] = cosine_dis(embedding.ix[train_types[i][4]],embedding.ix[train_types[i][5]])
    
    if(i%100==0):
        print(i)

train_words_dis_df = pd.DataFrame(train_words_dis)
train_words_dis_df.to_csv('train_words_dis.csv')



0


In [14]:
train_num = len(train_words); #训练量
train_words_dis = [([float]*nn_input_dim) for i in range(train_num)]
for i in range(0,train_num):
    '''
    train_words_dis[i][0] = cosine_dis(embedding.ix[train_words[i][0]],embedding.ix[train_words[i][1]])
    train_words_dis[i][1] = cosine_dis(embedding.ix[train_words[i][0]],embedding.ix[train_words[i][2]])
    train_words_dis[i][2] = cosine_dis(embedding.ix[train_words[i][0]],embedding.ix[train_words[i][3]])
    train_words_dis[i][3] = cosine_dis(embedding.ix[train_words[i][0]],embedding.ix[train_words[i][4]])
    train_words_dis[i][4] = cosine_dis(embedding.ix[train_words[i][0]],embedding.ix[train_words[i][5]])
    train_words_dis[i][5] = cosine_dis(embedding.ix[train_words[i][1]],embedding.ix[train_words[i][2]])
    train_words_dis[i][6] = cosine_dis(embedding.ix[train_words[i][1]],embedding.ix[train_words[i][3]])
    train_words_dis[i][7] = cosine_dis(embedding.ix[train_words[i][1]],embedding.ix[train_words[i][4]])
    train_words_dis[i][8] = cosine_dis(embedding.ix[train_words[i][1]],embedding.ix[train_words[i][5]])
    train_words_dis[i][9] = cosine_dis(embedding.ix[train_words[i][2]],embedding.ix[train_words[i][3]])
    train_words_dis[i][10] = cosine_dis(embedding.ix[train_words[i][2]],embedding.ix[train_words[i][4]])
    train_words_dis[i][11] = cosine_dis(embedding.ix[train_words[i][2]],embedding.ix[train_words[i][5]])
    train_words_dis[i][12] = cosine_dis(embedding.ix[train_words[i][3]],embedding.ix[train_words[i][4]])
    train_words_dis[i][13] = cosine_dis(embedding.ix[train_words[i][3]],embedding.ix[train_words[i][5]])
    train_words_dis[i][14] = cosine_dis(embedding.ix[train_words[i][4]],embedding.ix[train_words[i][5]])
    
    '''
    train_words_dis[i][0] = embedding.ix[train_words[i][0]]+embedding.ix[train_words[i][1]]
    train_words_dis[i][1] = embedding.ix[train_words[i][0]]+embedding.ix[train_words[i][2]]
    train_words_dis[i][2] = embedding.ix[train_words[i][0]]+embedding.ix[train_words[i][3]]
    train_words_dis[i][3] = embedding.ix[train_words[i][0]]+embedding.ix[train_words[i][4]]
    train_words_dis[i][4] = embedding.ix[train_words[i][0]]+embedding.ix[train_words[i][5]]
    
    if(i%1000==0):
        print(i)



0
1000


KeyError: '四千余'

In [11]:
train_words

[['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown'],
 ['unknown', 'unknown', 'unknown',

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0.186247,0.186247,0.186247,0.186247,0.186247,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1,0.186247,0.186247,0.186247,-0.111000,0.373484,1.000000,1.000000,0.490511,0.368714,1.000000,0.490511,0.368714,0.490511,0.368714,0.219010
2,2,-0.111000,0.490511,0.490511,0.490511,0.490511,0.186247,0.186247,0.186247,0.186247,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,3,1.000000,0.186247,0.186247,0.186247,0.186247,0.186247,0.186247,0.186247,0.186247,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,4,-0.111000,0.490511,0.490511,0.490511,1.000000,0.186247,0.186247,0.186247,-0.111000,1.000000,1.000000,0.490511,1.000000,0.490511,0.490511
5,5,1.000000,0.186247,0.186247,0.186247,-0.111000,0.186247,0.186247,0.186247,-0.111000,1.000000,1.000000,0.490511,1.000000,0.490511,0.490511
6,6,-0.111000,0.490511,0.490511,1.000000,0.219010,0.186247,0.186247,-0.111000,0.373484,1.000000,0.490511,0.368714,0.490511,0.368714,0.219010
7,7,-0.111000,0.490511,0.490511,1.000000,0.219010,0.186247,0.186247,-0.111000,0.373484,1.000000,0.490511,0.368714,0.490511,0.368714,0.219010
8,8,1.000000,0.186247,0.186247,-0.111000,0.373484,0.186247,0.186247,-0.111000,0.373484,1.000000,0.490511,0.368714,0.490511,0.368714,0.219010
9,9,0.186247,0.186247,0.186247,0.186247,-0.111000,1.000000,1.000000,1.000000,0.490511,1.000000,1.000000,0.490511,1.000000,0.490511,0.490511


In [19]:
aaa = np.zeros(100)

In [20]:
aaa

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [1]:
import pandas as pd
import numpy as np
import math
import theano
from sklearn import datasets
import pylab
import theano.tensor as T



In [9]:
aa = pd.DataFrame()

In [12]:
aa.ix[0][0]=1

IndexError: index 0 is out of bounds for axis 0 with size 0